In [856]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [857]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.royallepage.ca/en/search/homes/?search_str=Ontario%2C+ON%2C+CAN&csrfmiddlewaretoken=YSIjGH3mnehWiY3LUZubDuK5AfVsTC9MpkC2mnNbv9fTNiMml0UdFV4UINKH3JOL&property_type=7%2C8%2C5&house_type=102%2C103%2C104&features=&listing_type=recentlyCreated%2CopenHouseAvailable&lat=50.926163435000035&lng=-84.74492999999995&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=5&address=Ontario&method=homes&address_type=province&city_name=&prov_code=ON&school_id=&min_price=0&max_price=3000000&min_leaseprice=0&max_leaseprice=5000%2B&beds=0&baths=0&transactionType=SALE&archive_status=All&archive_timespan=6&sfproperty_type%5B7%5D=7&sfproperty_type%5B8%5D=8&sfproperty_type%5B5%5D=5&sflisting_type%5BrecentlyCreated%5D=recentlyCreated&sflisting_type%5BopenHouseAvailable%5D=openHouseAvailable&sfhouse_type%5B102%5D=102&sfhouse_type%5B103%5D=103&sfhouse_type%5B104%5D=104&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [858]:
# Go through all pages to collect data
house_listings = []
for x in range(1, 23):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    print('Gathering data from page:', x)
    for link in links:
        
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()

# Number of listings
len(house_listings)


Gathering data from page: 1
Gathering data from page: 2
Gathering data from page: 3
Gathering data from page: 4
Gathering data from page: 5
Gathering data from page: 6
Gathering data from page: 7
Gathering data from page: 8
Gathering data from page: 9
Gathering data from page: 10
Gathering data from page: 11
Gathering data from page: 12
Gathering data from page: 13
Gathering data from page: 14
Gathering data from page: 15
Gathering data from page: 16
Gathering data from page: 17
Gathering data from page: 18
Gathering data from page: 19
Gathering data from page: 20
Gathering data from page: 21
Gathering data from page: 22


920

In [859]:
browser.quit()

In [860]:
# Dependencies
import requests
import json

# Import the API key
from config import apiKey

# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 
    
    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text


    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    labels = building_feature_section.find_all('span', class_='label')
    values = building_feature_section.find_all('span', class_="value")
    

    wanted_features = ['Fireplace', 'Style', 'Cooling Type', 'Heating Type', 'Building Type']
    listed_features = []
    listed_values = []

    for i in range(len(labels)):
        listed_features.append(labels[i].text.strip(":"))
        listed_values.append(values[i].text.strip(":"))
        
    for i in range(len(wanted_features)):   
        if wanted_features[0] in listed_features:
            # Create index/value pairs for each feature
            fire_index = listed_features.index(wanted_features[0])
            fire_place = listed_values[fire_index]
            
        if wanted_features[1] in listed_features:
            style_index = listed_features.index(wanted_features[1])
            style_type = listed_values[style_index]
        
        if wanted_features[2] in listed_features:
            cooling_index = listed_features.index(wanted_features[2])
            cooling_type = listed_values[cooling_index]

        if wanted_features[3] in listed_features:
            heating_index = listed_features.index(wanted_features[3])
            heating_type = listed_values[heating_index]
            
        if wanted_features[4] in listed_features:    
            building_index = listed_features.index(wanted_features[4])
            building_type = listed_values[building_index]
        else:
            fire_place = ""
            style_place = ""
            cooling_type = ""
            heating_type = ""
            building_type = ""
            
        
    # Store in dictionary
    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    houses['cooling_type'] = cooling_type
    houses['heating_type'] = heating_type
    houses['fire_place'] = fire_place
    houses['building_type'] = building_type
    
    # Build target url with address and apikey
    string_search = address.split(",")[1:]
    new_string = (",").join(string_search).strip()                           
    
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={new_string}&format=json&apiKey={apiKey}"
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()

    # Get coordinate information
    lat = geo_data["results"][0]["lat"]
    lon = geo_data["results"][0]["lon"]
    
    houses['lat'] = lat
    houses['lon'] = lon
    
    # Store information of each house in dictionary 
    house_listing_info.append(houses)
    
browser.quit()

In [864]:
house_listing_info

[{'address': '6 HONEYBOURNE CRES, Markham, Ontario, L3P 1P3',
  'price': '$2,998,000',
  'number_of_beds': '5+1 Beds',
  'number_of_baths': '8 Baths',
  'style': 'Detached',
  'cooling_type': 'Central air conditioning',
  'heating_type': 'Forced air',
  'fire_place': 'Yes',
  'building_type': 'House',
  'lat': 43.8563707,
  'lon': -79.3376825},
 {'address': '83 BOARDWALK DR, Toronto, Ontario, M4L 3X9',
  'price': '$2,599,900',
  'number_of_beds': '3 Beds',
  'number_of_baths': '4 Baths',
  'style': 'Detached',
  'cooling_type': 'Central air conditioning',
  'heating_type': 'Forced air',
  'fire_place': 'Yes',
  'building_type': 'House',
  'lat': 43.67089,
  'lon': -79.316931},
 {'address': '2158 PINEVIEW DR, Oakville, Ontario, L6H 5M3',
  'price': '$1,999,000',
  'number_of_beds': '5 Beds',
  'number_of_baths': '4 Baths',
  'style': 'Detached',
  'cooling_type': 'Central air conditioning',
  'heating_type': 'Forced air',
  'fire_place': 'Yes',
  'building_type': 'House',
  'lat': 43.44

In [865]:
print(f"Rows of data to work with: {len(house_listing_info)}")

Rows of data to work with: 920


In [866]:
import csv

try:
    with open('./Resources/housing.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['lat', 'lon','address','style','building_type', 'cooling_type', 'heating_type', 'fire_place', 'price', 'number_of_beds', 'number_of_baths',])
        writer.writeheader()
        for data in house_listing_info:
            writer.writerow(data)
except IOError:
    print("I/O error")